In [11]:
import sys
sys.modules["multiprocessing"]

globals()["sys"]

<module 'sys' (built-in)>

In [6]:
import multiprocessing as mp
import time


def f(x):
    time.sleep(3)
    return x

pool = mp.Pool(processes=1, maxtasksperchild=1)
dgp_iterator = pool.imap(f, range(1, 6))
x = True
while True:
    try:
        x = dgp_iterator.next(timeout=1)
        print(x)
    except StopIteration:
        print("done")
        break
    except mp.TimeoutError:
        print("Timeout")
        
    
pool.close()
pool.join()

Timeout
Timeout
Timeout
1
Timeout
Timeout
Timeout
2
Timeout
Timeout
Timeout
3
Timeout
Timeout
Timeout
4
Timeout
Timeout
Timeout
5
done


In [7]:
from sympy.abc import x, y
from sympy.utilities.autowrap import autowrap, binary_function, ufuncify, CodeWrapper
import pickle
import importlib

In [8]:
# cc = ufuncify(list(expr.free_symbols), expr)
# cc = autowrap(expr, args=list(expr.free_symbols))

In [14]:
expr = x * y + 1
mod_name = f"mod_{abs(hash(expr))}_{CodeWrapper._module_counter}"
CodeWrapper.module_name = mod_name
cc = ufuncify(list(expr.free_symbols), expr, tempdir="./c_code")

# mod = importlib.import_module(mod_name)
# dir(mod)
# func_name = next(filter(lambda x: x.startswith("wrapped"), dir(mod)))
f_data = (mod_name, "autofunc_c")

In [4]:
import numpy as np
np.random.randint(1e7)

9184891

In [16]:
f_data

('mod_6206980849027394656_1', 'autofunc_c')

In [6]:
f_data = ('mod_1453896700186991053_0', 'autofunc_c')
mod = importlib.import_module(f_data[0])
func = getattr(mod, f_data[1])
func(np.array([2, 4, 5], dtype="float"), np.array([3, 5, 8], dtype="float"))

array([ 7., 21., 41.])

In [7]:
def compiler(expression):
    
    expression_val = None

    free_symbols = getattr(expression, "free_symbols", None)

    if free_symbols is not None:
        expr_func_ordered_symbols = list(free_symbols)

        mod_name = f"mod_{abs(hash(expression))}_{CodeWrapper._module_counter}"
        CodeWrapper.module_name = mod_name
        compiled_func = ufuncify(expr_func_ordered_symbols, expression)

        mod = importlib.import_module(mod_name)
        func_name = next(filter(lambda x: x.startswith("wrapped"), dir(mod)))

        expr_data = (mod_name, func_name)
        expression_args = [str(symbol) for symbol in expr_func_ordered_symbols]
    else:
        # No free symbols, return expression itself.
        self.expression_val = expression


In [8]:
compiler(expr)

In [17]:
import sys
list(filter(lambda x: x.startswith("mod_"), sys.modules.keys()))
mod = importlib.import_module("mod_4694703022551149035_1")
mod

<module 'mod_4694703022551149035_1' from '/tmp/tmp9u_5jtly_sympy_compile/mod_4694703022551149035_1.cpython-37m-x86_64-linux-gnu.so'>

In [19]:
dir(importlib)

['_RELOADING',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__import__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_bootstrap',
 '_bootstrap_external',
 '_imp',
 '_r_long',
 '_w_long',
 'abc',
 'find_loader',
 'import_module',
 'invalidate_caches',
 'machinery',
 'reload',
 'sys',
 'types',
 'util',
 'warnings']

In [11]:
import wrapper_module_1
wrapper_module_1.autofunc_c(1, 2)

ModuleNotFoundError: No module named 'wrapper_module_1'

In [ ]:
class A:
    pass

a = A()
isinstance(a, A)

In [15]:
!ls c_code

build							   setup.py
mod_4694703022551149035_6.c				   wrapped_code_0.c
mod_4694703022551149035_6.cpython-37m-x86_64-linux-gnu.so  wrapped_code_0.h
mod_6206980849027394656_0.c				   wrapped_code_1.c
mod_6206980849027394656_0.cpython-37m-x86_64-linux-gnu.so  wrapped_code_1.h
mod_6206980849027394656_1.c				   wrapped_code_6.c
mod_6206980849027394656_1.cpython-37m-x86_64-linux-gnu.so  wrapped_code_6.h


In [ ]:
import importlib.util
spec = importlib.util.spec_from_file_location("mod_6206980849027394656_1", "./c_code/mod_6206980849027394656_1.cpython-37m-x86_64-linux-gnu.so")
foo = importlib.util.module_from_spec(spec)
spec.loader.exec_module(foo)
dir(foo)

In [1]:
import sys
sys.path.append('./c_code')
import mod_6206980849027394656_1
mod_6206980849027394656_1

<module 'mod_6206980849027394656_1' from './c_code/mod_6206980849027394656_1.cpython-37m-x86_64-linux-gnu.so'>

In [3]:
class A:
    def __init__(self, a, b):
        (self.a, self.b) = (a, b)

In [7]:
a = A(3, 2)
a.a

3